# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [3]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/Users/miwa/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-041: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前日からの売上金額増減を計算せよ。なお、計算結果は10件表示すればよい。

In [11]:
# 参考
# https://note.nkmk.me/python-pandas-diff-pct-change/

df_tmp = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_tmp['diff'] = df_tmp['amount'].diff()
df_tmp.head(10)

,sales_ymd,amount,diff
0,20170101,33723,NaN
1,20170102,24165,-9558.0
2,20170103,27503,3338.0
3,20170104,36165,8662.0
4,20170105,37830,1665.0
5,20170106,32387,-5443.0
6,20170107,23415,-8972.0
7,20170108,24737,1322.0
8,20170109,26718,1981.0
9,20170110,20143,-6575.0


---
> P-042: レシート明細データフレーム（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、１日前、２日前、３日前のデータを結合せよ。結果は10件表示すればよい。

In [17]:
# 参考


df_tmp = df_receipt.groupby('sales_ymd').amount.sum().reset_index()
df_tmp['1日前'] = df_tmp['amount'] - df_tmp['amount'].diff()
df_tmp['2日前'] = df_tmp['amount'] - df_tmp['amount'].diff(2)
df_tmp['3日前'] = df_tmp['amount'] - df_tmp['amount'].diff(3)
df_tmp.head(10)

,sales_ymd,amount,1日前,2日前,3日前
0,20170101,33723,NaN,NaN,NaN
1,20170102,24165,33723.0,NaN,NaN
2,20170103,27503,24165.0,33723.0,NaN
3,20170104,36165,27503.0,24165.0,33723.0
4,20170105,37830,36165.0,27503.0,24165.0
5,20170106,32387,37830.0,36165.0,27503.0
6,20170107,23415,32387.0,37830.0,36165.0
7,20170108,24737,23415.0,32387.0,37830.0
8,20170109,26718,24737.0,23415.0,32387.0
9,20170110,20143,26718.0,24737.0,23415.0


---
> P-043： レシート明細データフレーム（df_receipt）と顧客データフレーム（df_customer）を結合し、性別（gender）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータフレーム（df_sales_summary）を作成せよ。性別は0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [5]:
# できなかったので答えみた
# 参考：https://note.nkmk.me/python-pandas-map-applymap-apply/
# 参考：https://note.nkmk.me/python-pandas-pivot-table/

df_tmp = pd.merge(df_receipt, df_customer, how ='inner', on="customer_id")
df_tmp['era'] = df_tmp['age'].apply(lambda x: math.floor(x / 10) * 10)
df_sales_summary = pd.pivot_table(df_tmp, index='era', columns='gender_cd', values='amount', aggfunc='sum').reset_index()
df_sales_summary.columns = ['era', 'male', 'female', 'unknown']
df_sales_summary

,era,male,female,unknown
0,10,1591.0,149836.0,4317.0
1,20,72940.0,1363724.0,44328.0
2,30,177322.0,693047.0,50441.0
3,40,19355.0,9320791.0,483512.0
4,50,54320.0,6685192.0,342923.0
5,60,272469.0,987741.0,71418.0
6,70,13435.0,29764.0,2427.0
7,80,46360.0,262923.0,5111.0
8,90,NaN,6260.0,NaN


---
> P-044： 前設問で作成した売上サマリデータフレーム（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータフレームから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を'00'、女性を'01'、不明を'99'とする。

In [11]:
# ここもわからなかったので答えみた
# 参考：https://note.nkmk.me/python-pandas-stack-unstack-pivot/

df_tmp = df_sales_summary.set_index('era').stack().reset_index()
# .stack().reset_index().replace({'female':'01','male':'00','unknown':'09'}).rename(columns={'level_1':'gender_cd', 0: 'amount'})
df_tmp

,era,level_1,0
0,10,male,1591.0
1,10,female,149836.0
2,10,unknown,4317.0
3,20,male,72940.0
4,20,female,1363724.0
5,20,unknown,44328.0
6,30,male,177322.0
7,30,female,693047.0
8,30,unknown,50441.0
9,40,male,19355.0


---
> P-045: 顧客データフレーム（df_customer）の生年月日（birth_day）は日付型（Date）でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに抽出せよ。データは10件を抽出すれば良い。

In [24]:
# 答えみた
# 参考：https://note.nkmk.me/python-pandas-datetime-timestamp/

pd.concat([df_customer['customer_id'], pd.to_datetime(df_customer['birth_day']).dt.strftime('%Y%m%d')], axis = 1).head(10)

,customer_id,birth_day
0,CS021313000114,19810429
1,CS037613000071,19520401
2,CS031415000172,19761004
3,CS028811000001,19330327
4,CS001215000145,19950329
5,CS020401000016,19740915
6,CS015414000103,19770809
7,CS029403000008,19730817
8,CS015804000004,19310502
9,CS033513000180,19620711
